In [1]:
from sklearn.cluster import KMeans
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler,OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
import pandas as pd

In [5]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample  = pd.read_csv('sample_submission.csv')
train.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [11]:
#transforming categorical columns to encoded values
categorical_columns = ['Surname','Geography','Gender']
en = OrdinalEncoder()
en.fit(train[categorical_columns])
train_transform = en.transform(train[categorical_columns])
en.fit(test[categorical_columns])
test_transform = en.transform(test[categorical_columns])
test[categorical_columns] = test_transform
train[categorical_columns] = train_transform

In [12]:
train.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,1924.0,668,0.0,1.0,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,1925.0,627,0.0,1.0,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,1178.0,678,0.0,1.0,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,1299.0,581,0.0,1.0,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,467.0,716,2.0,1.0,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [13]:
train.describe()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,165034.0000,1.650340e+05,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000
mean,82516.5000,1.569201e+07,1547.281948,656.454373,0.648545,0.564429,38.125888,5.020353,55478.086689,1.554455,0.753954,0.497770,112574.822734,0.211599
std,47641.3565,7.139782e+04,772.771233,80.103340,0.816574,0.495833,8.867205,2.806159,62817.663278,0.547154,0.430707,0.499997,50292.865585,0.408443
min,0.0000,1.556570e+07,0.000000,350.000000,0.000000,0.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,11.580000,0.000000
25%,41258.2500,1.563314e+07,924.000000,597.000000,0.000000,0.000000,32.000000,3.000000,0.000000,1.000000,1.000000,0.000000,74637.570000,0.000000
50%,82516.5000,1.569017e+07,1589.000000,659.000000,0.000000,1.000000,37.000000,5.000000,0.000000,2.000000,1.000000,0.000000,117948.000000,0.000000
75%,123774.7500,1.575682e+07,2186.000000,710.000000,1.000000,1.000000,42.000000,7.000000,119939.517500,2.000000,1.000000,1.000000,155152.467500,0.000000
max,165033.0000,1.581569e+07,2796.000000,850.000000,2.000000,1.000000,92.000000,10.000000,250898.090000,4.000000,1.000000,1.000000,199992.480000,1.000000


In [14]:
X = train.drop(columns = ['Exited','CustomerId'])
y = train['Exited']
X_train, X_test,y_train, y_test = train_test_split(X,y , random_state= 42)

In [15]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('kmeans', KMeans(n_clusters=3, random_state=42)),
    ('selector', SelectFromModel(estimator=RandomForestClassifier(n_estimators=100, random_state=42))),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

In [17]:
prediction = pipeline.fit(X,y)
# accuracy = cross_val_score(pipeline,X,y,cv = 5)
# accuracy.mean()

C:\Users\ACER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [8]:
pipeline.predict_proba(X_test)

array([[0.94455733, 0.05544267],
       [0.87419334, 0.12580666],
       [0.96352922, 0.03647078],
       ...,
       [0.78980349, 0.21019651],
       [0.89182768, 0.10817232],
       [0.76652136, 0.23347864]])

In [9]:
pipeline.fit(X,y)

Pipeline(steps=[('norml', StandardScaler()), ('clf', LogisticRegression())])

In [10]:
prob = pipeline.predict_proba(test)
pred = pipeline.predict(test)
sample['Exited'] = prob[:,1]

In [11]:
sample.to_csv('submission.csv', index = False)